In [1]:
import pandas as pd
import chardet as chardet

/var/folders/gm/c4sdklyj1d5bmsf5thm4k8f00000gn/T/ipykernel_26166/204012549.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


# GPA Data Cleaning

In [2]:
# collecting and formatting lines:
input_file = 'gpa.csv' # this is encoded in UTF-16
lines = []
with open(input_file, 'r', encoding='utf-16') as infile:
    for line in infile:
        fields = line.strip().split(',')
        if len(fields) != 2:
            fields = [','.join(fields[:2])] + fields[2:]
        lines.append(','.join(fields))

# writing to a new tsv file:
output_file = 'gpa_utf8.csv'
with open(output_file, 'w', encoding='utf-8') as outfile:
    for line in lines:
        outfile.write(line + '\n')

In [3]:
gpa_df = pd.read_csv("gpa_utf8.csv", sep="\t")

In [4]:
# removing redundant cells and columns
gpa_df_usa = gpa_df[gpa_df["City"] != 'n/a ']
gpa_df_usa = gpa_df_usa.drop(columns=["Calculation1"])

In [5]:
# creating an enrolled GPA only DF
enrl_gpa_df_usa = gpa_df_usa[gpa_df_usa["Measure Names"] == "Enrl GPA"]
enrl_gpa_df_usa = enrl_gpa_df_usa.drop(columns = ["Measure Names"])
enrl_gpa_df_usa = enrl_gpa_df_usa.rename(columns={"Measure Values": "Enrl GPA"}).set_index("School")

# creating an admitted GPA only DF
adm_gpa_df_usa = gpa_df_usa[gpa_df_usa["Measure Names"] == "Adm GPA"]
adm_gpa_df_usa = adm_gpa_df_usa.drop(columns = ["Measure Names"])
adm_gpa_df_usa = adm_gpa_df_usa.rename(columns={"Measure Values": "Adm GPA"}).set_index("School")

In [6]:
# creating a DF with both enrolled and admitted GPA
both_gpa_df_usa = enrl_gpa_df_usa.copy()
both_gpa_df_usa["Adm GPA"] = adm_gpa_df_usa["Adm GPA"]

In [7]:
# assigning column to the newest DF that contains GPA difference
def calc_gpa_diff(ser):
    return ser["Adm GPA"] - ser["Enrl GPA"]
both_gpa_df_usa["GPA Diff"] = both_gpa_df_usa.apply(calc_gpa_diff, axis=1)
both_gpa_df_usa["GPA Diff"]

School
A & M CONSOLIDATED HIGH SCHOOL         NaN
A B MILLER HIGH SCHOOL                 NaN
A N MCCALLUM HIGH SCHOOL               NaN
ABINGTON FRIENDS SCHOOL                NaN
ABRAHAM LINCOLN HIGH SCHOOL       0.001727
                                    ...   
YUCAIPA SENIOR HIGH SCHOOL             NaN
YUCCA VALLEY HIGH SCHOOL               NaN
YULA BOYS HIGH SCHOOL                  NaN
YUMA CATHOLIC HIGH SCHOOL              NaN
ZIONSVILLE COMMUNITY HS                NaN
Name: GPA Diff, Length: 2759, dtype: float64

In [8]:
both_gpa_df_usa

,City,County/State/Country,Enrl GPA,Adm GPA,GPA Diff
School,,,,,
A & M CONSOLIDATED HIGH SCHOOL,College Station,Texas,NaN,NaN,NaN
A B MILLER HIGH SCHOOL,Fontana,San Bernardino,NaN,4.090000,NaN
A N MCCALLUM HIGH SCHOOL,Austin,Texas,NaN,4.228333,NaN
ABINGTON FRIENDS SCHOOL,Jenkintown,Pennsylvania,NaN,NaN,NaN
ABRAHAM LINCOLN HIGH SCHOOL,Los Angeles,Los Angeles,4.081176,4.082903,0.001727
...,...,...,...,...,...
YUCAIPA SENIOR HIGH SCHOOL,Yucaipa,San Bernardino,NaN,4.274615,NaN
YUCCA VALLEY HIGH SCHOOL,Yucca Valley,San Bernardino,NaN,NaN,NaN
YULA BOYS HIGH SCHOOL,Los Angeles,Los Angeles,NaN,NaN,NaN


# Admittance Data Cleaning

In [9]:
# collecting and formatting lines:
input_file = 'admittance.csv'
lines = []
with open(input_file, 'r', encoding='utf-16') as infile:
    for line in infile:
        fields = line.strip().split(',')
        if len(fields) != 2:
            fields = [','.join(fields[:2])] + fields[2:]
        lines.append(','.join(fields))

# writing to a new .tsv file
output_file = 'admittance_utf8.csv'
with open(output_file, 'w', encoding='utf-8') as outfile:
    for line in lines:
        outfile.write(line + '\n')

In [10]:
admittance_df = pd.read_csv("admittance_utf8.csv", sep="\t")
admittance_df.head()

,Calculation1,City,County/State/ Territory,Count,gender,School,Pivot Field Values
0,2ND NATURE ACADEMY HIGH SCHOOL300024,Nashua,NH,Enr,Unknown,2ND NATURE ACADEMY HIGH SCHOOL,NaN
1,2ND NATURE ACADEMY HIGH SCHOOL300024,Nashua,NH,Adm,Unknown,2ND NATURE ACADEMY HIGH SCHOOL,NaN
2,2ND NATURE ACADEMY HIGH SCHOOL300024,Nashua,NH,App,Unknown,2ND NATURE ACADEMY HIGH SCHOOL,NaN
3,2ND NATURE ACADEMY HIGH SCHOOL300024,Nashua,NH,Enr,Other,2ND NATURE ACADEMY HIGH SCHOOL,NaN
4,2ND NATURE ACADEMY HIGH SCHOOL300024,Nashua,NH,Adm,Other,2ND NATURE ACADEMY HIGH SCHOOL,NaN


In [11]:
# removing redundant cells and columns
admittance_df_usa = admittance_df[admittance_df["City"] != 'n/a ']
admittance_df_usa = admittance_df[admittance_df["City"].isna() == False]
admittance_df_usa = admittance_df_usa.drop(columns=["Calculation1"])
admittance_df_usa = admittance_df_usa[admittance_df["gender"] == "All"]

/var/folders/gm/c4sdklyj1d5bmsf5thm4k8f00000gn/T/ipykernel_26166/1595058900.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  admittance_df_usa = admittance_df_usa[admittance_df["gender"] == "All"]


In [12]:
# creating an enrolled GPA only DF:
enr_admittance_df_usa = admittance_df_usa[admittance_df_usa["Count"] == "Enr"].set_index("School")

# creating an admitted GPA only DF:
adm_admittance_df_usa = admittance_df_usa[admittance_df_usa["Count"] == "Adm"].set_index("School")

# creating an applied GPA only DF:
app_admittance_df_usa = admittance_df_usa[admittance_df_usa["Count"] == "App"].set_index("School")

# creating a DF with all three GPAs:
final_admittance_df_usa = app_admittance_df_usa.copy()
final_admittance_df_usa = final_admittance_df_usa.rename(columns={"Pivot Field Values": "App Ct"})
final_admittance_df_usa["Adm Ct"] = adm_admittance_df_usa["Pivot Field Values"]
final_admittance_df_usa["Enr Ct"] = enr_admittance_df_usa["Pivot Field Values"]

# dropping redundant columns:
final_admittance_df = final_admittance_df_usa.drop(columns=["Count","gender"])

In [15]:
final_admittance_df.head()

,City,County/State/ Territory,App Ct,Adm Ct,Enr Ct
School,,,,,
2ND NATURE ACADEMY HIGH SCHOOL,Nashua,NH,NaN,NaN,NaN
21ST CENTURY CYBER CHARTER SCH,West Chester,PA,NaN,NaN,NaN
A & M CONSOLIDATED HIGH SCHOOL,College Station,TX,5.0,3.0,NaN
A B MILLER HIGH SCHOOL,Fontana,San Bernardino,32.0,13.0,NaN
A BEKA ACADEMY VIDEO FOR HOMES,Pensacola,FL,NaN,NaN,NaN


In [17]:
def in_or_out(row):
    if len(row["County/State/ Territory"]) <= 2:
        return "Out-of-State"
    else:
        return "In-State"
        
final_admittance_df["IS/OOS"] = final_admittance_df.apply(lambda row: in_or_out(row), axis=1)
final_admittance_df

,City,County/State/ Territory,App Ct,Adm Ct,Enr Ct,IS/OOS
School,,,,,,
2ND NATURE ACADEMY HIGH SCHOOL,Nashua,NH,NaN,NaN,NaN,Out-of-State
21ST CENTURY CYBER CHARTER SCH,West Chester,PA,NaN,NaN,NaN,Out-of-State
A & M CONSOLIDATED HIGH SCHOOL,College Station,TX,5.0,3.0,NaN,Out-of-State
A B MILLER HIGH SCHOOL,Fontana,San Bernardino,32.0,13.0,NaN,In-State
A BEKA ACADEMY VIDEO FOR HOMES,Pensacola,FL,NaN,NaN,NaN,Out-of-State
...,...,...,...,...,...,...
YUCAIPA SENIOR HIGH SCHOOL,Yucaipa,San Bernardino,49.0,13.0,4.0,In-State
YUCCA VALLEY HIGH SCHOOL,Yucca Valley,San Bernardino,14.0,NaN,NaN,In-State
YULA BOYS HIGH SCHOOL,Los Angeles,Los Angeles,11.0,3.0,NaN,In-State


In [18]:
final_admittance_df["County/State/ Territory"].unique()

array(['NH', 'PA', 'TX', 'San Bernardino', 'FL', 'WA', 'AK', 'AZ', 'MD',
       'San Joaquin', 'NY', 'Los Angeles', 'San Francisco', 'Santa Clara',
       'Orange', 'NJ', 'PR', 'SC', 'MA', 'IL', 'CT', 'NV', 'MN',
       'Riverside', 'GU', 'Humboldt', 'LA', 'San Diego', 'Contra Costa',
       'MO', 'WI', 'Ventura', 'MI', 'CO', 'HI', 'Alameda', 'Sacramento',
       'AL', 'NM', 'GA', 'VA', 'Monterey', 'OK', 'OR', 'UT',
       'Santa Barbara', 'Santa Cruz', 'Amador', 'ID', 'Napa', 'IA', 'MS',
       'Sonoma', 'Shasta', 'Mendocino', 'KS', 'OH', 'Solano', 'VI', 'NC',
       'San Mateo', 'DC', 'Marin', 'DE', 'AR', 'TN', 'Kings',
       'San Luis Obispo', 'Kern', 'Stanislaus', 'KY', 'Placer', 'Merced',
       'IN', 'RI', 'Nevada', 'ME', 'MT', 'Inyo', 'VT', 'Imperial',
       'Calaveras', 'WV', 'SD', 'NE', 'WY', 'Fresno', 'Tulare', 'ND',
       'El Dorado', 'Plumas', 'Butte', 'Madera', 'Yolo', 'Lake', 'Mono',
       'Colusa', 'Tehama', 'Del Norte', 'Sierra', 'Sutter', 'Siskiyou',
       'AS', '